# Procesamiento de lenguaje natural
En este capítulo se verá como procesar lenguaje natural haciendo uso de la librería/herramienta *spaCy*.

En lugar de transformar, spaCy conserva el texto original a lo largo de todo el proceso, añadiendo nuevas capas de información al mismo.

Se irá mostrando paso a paso como se hace uso de las distintas herramientas que proporciona la librería con ejemplos sencillos, para después finalizar con todo el proceso aplicado sobre el Data Frame que se ha ido creando en los capítulos anteriores.

In [2]:
import sys, os

#Carga del archivo setup.py
%run -i ../pyenv_settings/setup.py

#Imports y configuraciones de gráficas
%run "$BASE_DIR/pyenv_settings/settings.py"

#Reset del entorno virtual al iniciar la ejecución
#%reset -f

%reload_ext autoreload
%autoreload 0
%config InlineBackend.figure_format = 'png'

# to print output of all statements and not just the last
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# otherwise text between $ signs will be interpreted as formula and printed in italic
pd.set_option('display.html.use_mathjax', False)

You are working on a local system.
Files will be searched relative to "..".


## PNL utilizando *spaCy*

### Antes de utilizar spaCy
A la hora de instalar el modelo el usuario se debe asegurar de que algunas herramientas y librerías estén actualizadas a su última versión y que sean compatibles con la versión de python que se tiene instalada. En mi caso cuento con Python 3.12.7, y he tenido que actualizar con el comando *"pip install --upgrade nombre-paquete"*: *pip*, *setuptools*, *wheel*, *spacy*, *pydantic*, *thic*, y asegurarme de que las versiones eran compatibles entre ellas, por ejemplo, para thic tuve que instalar la versión 8.3.0 para asegurar la compatibilidad.

Una vez hecho todo esto, ya se puede instalar el modelo desde terminal (en este caso la del entorno virtual de python) con el comando:
*python -m spacy download en_core_web_sm*

Si aún con todo esto se siguen generando errores, como es mi caso, se recomienda crear un nuevo entorno virtual con una versión de más estable, como la 3.10 o 3.11, para asegurar la compatibilidad.

Para poder contar con distintas versiones de Python, es necesario tener instalado el paquete *pyenv* en el sistema. Una vez instalado, es posible instalar la versión deseada, 3.10.9 en mi caso. A partir de este modelo, se creará un entorno virtual utilizando pyenv e indicando la versión que se desea utilizar a la hora de definirlo.

### Instanciación de un pipeline
Se va a utilizar un modelo de procesamiento preentrenado, para luego instanciar un pipeline que nos servirá a lo largo del capítulo.

Cabe recalcar que el modelo utilizado, *en_core_web_sm*, debe ser descargado manualmente antes de utilizarse, en caso contrario no se cargará y el programa lanzará un error.

En la variable "nlp" se almacenará el objeto *Language*, el cuál contiene el vocabulario, el modelo y el pipeline de procesamiento.

In [3]:
import spacy

#Carga del modelo
nlp = spacy.load('en_core_web_sm')

#Instanciación del pipeline
nlp.pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x70c8b4a5e080>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x70c8b4a5e680>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x70c8b4cb7ed0>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x70c8b49a03c0>),
 ('lemmatizer',
  <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x70c8b487d840>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x70c8b4cb7df0>)]

El tokenizador de spaCy es bastante rápido, pero el resto de tareas no. Es por ello que si se desea analizar un dataset considerablemente extenso, se recomienda desactivar algunas funciones del modelo para que la duración disminuya significativamente.

En este caso, se va a desactivar *parser* y *named-entity recognition* porque se va a hacer uso, por ahora, del tokenizador y el *part-of-speech tagger*, etiquetador de partes del discurso, el cuál se explicará más adelante.

In [5]:
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
nlp.pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x70c8b3f867a0>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x70c8b48b47c0>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x70c8b3f06bc0>),
 ('lemmatizer',
  <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x70c8b3f06b00>)]

### Procesamiento de texto
En la llamada al pipeline nlp, este devuelve un objeto de tipo *spacy.tokens.doc.Doc* que contiene el acceso a los tokens, spans (rangos de tokens), y algunas anotaciones sobre el token.

In [15]:
text = "My friend is a professional race car drive, I would like to experience that kind of feeling"
doc = nlp(text)

for token in doc:
    print(token, end="|")

My|friend|is|a|professional|race|car|drive|,|I|would|like|to|experience|that|kind|of|feeling|

Se definirá a continuación una función que genera una tabla que contenga todos los tokens y sus atributos. El resultado se podría definir como un DataFrame en el que se puede utilizar la posición de cada elemento (token) como índice.

In [16]:
def display_nlp(doc, include_punct=False):
    #Generación del DataFrame con los tokens
    rows = []
    for i, t in enumerate(doc):
        if not t.is_punct or include_punct:
            row = {'token': i,  'text': t.text, 'lemma_': t.lemma_, 
                   'is_stop': t.is_stop, 'is_alpha': t.is_alpha,
                   'pos_': t.pos_, 'dep_': t.dep_, 
                   'ent_type_': t.ent_type_, 'ent_iob_': t.ent_iob_}
            rows.append(row)
    
    df = pd.DataFrame(rows).set_index('token')
    df.index.name = None
    
    return df

Cuando visualizamos el Data Frame creado, se observa que hay una columna con el atributo "is_stop" que indica si se trata de una *stop word* o no sin necesidad de usar un diccionario como se vió en capítulos anteriores gracias al uso del modelo ya preentrenado.

In [17]:
display_nlp(doc)

,text,lemma_,is_stop,is_alpha,pos_,dep_,ent_type_,ent_iob_
0,My,my,True,True,PRON,,,
1,friend,friend,False,True,NOUN,,,
2,is,be,True,True,AUX,,,
3,a,a,True,True,DET,,,
4,professional,professional,False,True,ADJ,,,
5,race,race,False,True,NOUN,,,
6,car,car,False,True,NOUN,,,
7,drive,drive,False,True,NOUN,,,
9,I,I,True,True,PRON,,,
10,would,would,True,True,AUX,,,


### Personalización del tokenizador
Debido a que la gran mayoría del texto que se va a desear analizar será inglés, hay ciertas palabras o expresiones que deben tomarse en cuenta para que el tokenizador no las elimine o separe. Este es el caso de palabras compuestas que están unidas con guión o guión bajo, expresiones o palabras que se inician con un "#" que puede dar un mayor contexto al texto, etc.

Se va a definir una función que, haciendo uso del tokenizador de spacy, lo modifica al mismo tiempo para que incluya este tipo de formaciones.

In [18]:
from spacy.tokenizer import Tokenizer
from spacy.util import compile_prefix_regex, \
                       compile_infix_regex, compile_suffix_regex

def custom_tokenizer(nlp):
    
    # use default patterns except the ones matched by re.search
    prefixes = [pattern for pattern in nlp.Defaults.prefixes 
                if pattern not in ['-', '_', '#']]
    suffixes = [pattern for pattern in nlp.Defaults.suffixes
                if pattern not in ['_']]
    infixes  = [pattern for pattern in nlp.Defaults.infixes
                if not re.search(pattern, 'xx-xx')]

    return Tokenizer(vocab          = nlp.vocab, 
                     rules          = nlp.Defaults.tokenizer_exceptions,
                     prefix_search  = compile_prefix_regex(prefixes).search,
                     suffix_search  = compile_suffix_regex(suffixes).search,
                     infix_finditer = compile_infix_regex(infixes).finditer,
                     token_match    = nlp.Defaults.token_match)

In [19]:
text = "@Pete: choose low-carb #food #eat-smart. _url_ ;-) 😋👍"

#nlp = spacy.load('en_core_web_sm')
nlp.tokenizer = custom_tokenizer(nlp)

doc = nlp(text)
for token in doc:
    print(token, end="|")

@Pete|:|choose|low-carb|#food|#eat-smart|.|_url_|;-)|😋|👍|